## Running pbilby on a cluster (or laptop)

This tutorial will demonstrate how to set up parallel_bilby (pbilby) jobs, which can be run on a cluster, via slurm, or on a laptop (though the latter will be limited to the number of available cores on the machine).

There are three steps to running pbilby. The first is the ini file. This contains everything to set up the run. The ini file `GW150914.ini` is contained in this directory.

The ini file is set up to analyze GW150914 using GWOSC data. We will use the `dynesty` nested sampling algorithm. Let's start by understanding the ini file. None of the cells detailing the ini file need to be run. I've just added them here to describe them in detail.



### ini file

The first part of the ini file sets the data specific settings

In [ ]:

################################################################################
####### Data generation arguments
################################################################################

trigger_time = 1126259462.4

################################################################################
####### Detector arguments
################################################################################

detectors = [H1, L1]
psd_dict = {H1=psd_data/h1_psd.txt, L1=psd_data/l1_psd.txt}
channel_dict = {H1:GWOSC, L1:GWOSC}
duration = 4

The trigger time is the time of the merger, as estimated by the search pipelines. This can be found from GWOSC. By convention, the data is such that the trigger time occurs 2s before the end of the data segment.

Next, we specify the detectors, PSD, data channel and data duration. Only LIGO Hanford and Livingston were operational at the time of GW150914 so we only specify these instruments. The PSD is contained in the `psd_data` directory. Because we are using GWOSC data, we only need to specify the data channel as GWOSC. Lastly, because GW150914 is only in band for a short duration (a few hundred ms) we will only analyze 4s of data containing the signal.

The next set of arguments set up the likelihood and prior:

In [ ]:
################################################################################
## Likelihood arguments
################################################################################

distance-marginalization=True
phase-marginalization=True
time-marginalization=True

################################################################################
## Prior arguments
################################################################################

prior-dict = {
  mass_ratio = Uniform(name='mass_ratio', minimum=0.125, maximum=1),
  chirp_mass = Uniform(name='chirp_mass', minimum=25, maximum=31),
  mass_1 = Constraint(name='mass_1', minimum=10, maximum=80),
  mass_2 = Constraint(name='mass_2', minimum=10, maximum=80),
  a_1 = Uniform(name='a_1', minimum=0, maximum=0.99),
  a_2 = Uniform(name='a_2', minimum=0, maximum=0.99),
  tilt_1 = Sine(name='tilt_1'),
  tilt_2 = Sine(name='tilt_2'),
  phi_12 = Uniform(name='phi_12', minimum=0, maximum=2 * np.pi, boundary='periodic'),
  phi_jl = Uniform(name='phi_jl', minimum=0, maximum=2 * np.pi, boundary='periodic'),
  luminosity_distance = PowerLaw(alpha=2, name='luminosity_distance', minimum=50, maximum=2000),
  dec =  Cosine(name='dec'),
  ra =  Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
  theta_jn =  Sine(name='theta_jn'),
  psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),
  phase =  Uniform(name='phase', minimum=0, maximum=2 * np.pi, boundary='periodic')
}


The likelihood arguments are flags that specify if the three parameters `distance, phase, time` should be numerically/analytically marginalized over each time the likelihood is called. Setting these to True can significantly speed up the run, and these parameters can be recovered in postprocessing (i.e., they're not lost if you choose to marginalized over them). The only time when you might not want to set these to true is if you're going to use a waveform that contains higher order mode content. In this case, the prescription for phase marginalization is formally invalid and phase-marginalization should be set to false.

Next we set the prior. These priors are chosen to be wide enough to capture the bulk of the posterior, but narrow enough so the run converges fairly quickly. 

The next set of arguments set the template waveform. Here we will use `IMRPhenomPv2` because it achieves a good trade off between accuracy and speed. More contemporary waveform models could easily be swapped in, e.g., `IMRPhenomXPHM`. 

In [ ]:
################################################################################
## Waveform arguments
################################################################################

waveform_approximant = IMRPhenomPv2
frequency-domain-source-model = lal_binary_black_hole

Next, we set up dynesty:

In [ ]:
###############################################################################
## Sampler settings
################################################################################

sampler = dynesty
nlive = 1000
nact = 5


These settings should be fine for a "quick" run, though our recommended settings for "production" analyses are `nlive=1500`, `nact=10`

Lastly, we set up the slurm scheduler:
    

In [ ]:
################################################################################
## Slurm Settings
################################################################################

nodes = 10
ntasks-per-node = 16
time = 24:00:00

The actual settings you choose will depend entirely on the cluster you run on. Here `ntasks-per-node` is the number of CPUs per node (or cores per node), so this job would specify 10 nodes, each with 16 cores, for a total of 160 cores/CPUs. 

## Running pbilby

Running pbilby is a two step process. First, we run `parallel_bilby_generation`. This creates the run directory structure, produces the `data_dump` file which contains the data, psd etc..., as well as the slurm submit script:

In [2]:
!parallel_bilby_generation GW150914.ini

14:01 bilby_pipe WARNING : You do not have nds2 (python-nds2-client) installed. You may  experience problems accessing interferometer data.
14:01 bilby_pipe WARNING : You do not have LDAStools.frameCPP (python-ldas-tools-framecpp) installed. You may experience problems accessing interferometer data.
14:01 bilby_pipe INFO    : Command line arguments: Namespace(Tmax=10000, accounting=None, adapt=False, autocorr_c=5.0, autocorr_tol=50.0, bilby_zero_likelihood_mode=False, burn_in_nact=50.0, calibration_model=None, catch_waveform_errors=False, channel_dict='{H1:GWOSC, L1:GWOSC}', check_point_deltaT=3600, clean=False, cluster=None, coherence_test=False, convert_to_flat_in_component_mass=False, create_plots=False, create_summary=False, data_dict=None, data_dump_file=None, data_format=None, default_prior='BBHPriorDict', deltaT=0.2, detectors=['H1', 'L1'], distance_marginalization=True, distance_marginalization_lookup_table=None, dlogz=0.1, do_not_save_bounds_in_resume=False, duration=4.0, dyne

When this runs successfully, you should see 
```
14:04 bilby INFO    : Complete ini written: outdir/GW150914_config_complete.ini
14:04 bilby INFO    : Setup complete, now run:
$ bash outdir/submit/bash_GW150914.sh
```

If you now inspect the directory, you'll see a new folder called `outdir`. This is where results, logs, data, and submit files are contained.

The last thing to do is to run parallel bilby. If you're running on a cluster, the easiest thing to do at this point would be to run `bash outdir/submit/bash_GW150914.sh`. However, let's first take a look at the contenets of `outdir/submit/bash_GW150914.sh`. The bash script contains instructions to run another script, `analysis_GW150914_0.sh`. Inside `analysis_GW150914_0.sh` is the *actual* command that's submitted by the slurm scheduler:

`mpirun parallel_bilby_analysis outdir/data/GW150914_data_dump.pickle --label GW150914_0 --outdir /Users/rsmi0016/git/parallel_bilby/examples/GW150914_IMRPhenomPv2/outdir/result --sampling-seed 1234`


This can be run on your laptop, or headnode to test if everything is working.